In [ ]:
import geopandas as gpd
from pyproj import Proj, transform
from shapely.geometry import Point
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import contextily as ctx
from pyproj import Proj, transform
import matplotlib.pyplot as plt
from scipy.stats import kurtosis, skew
import dask.dataframe as dd
from rasterio.transform import from_origin
import numpy as np
import rasterio
from shapely.ops import unary_union
from inequality.theil import Theil

In [ ]:
local_crs = 27700
place = "test"
lat = 55.86421405612109
lng = -4.251846930489373
country = "UK"
crs=4326
radius=1

In [ ]:
local_crs = 27700
place = "Glasgow"
lat = 55.86421405612109
lng = -4.251846930489373
country = "UK"
crs=4326
radius=100


In [ ]:
if lng:
    lon = lng

In [ ]:
lonlat=(lon, lat)

In [ ]:
buildings = gpd.read_parquet(f"../output/{place}/buildings_morphometric_p3.pq")
tessellation = gpd.read_parquet(f"../output/{place}/tessellation_morphometric_p3.pq")
edges = gpd.read_parquet(f"../output/{place}/edges_p4.pq")
nodes = gpd.read_parquet(f"../output/{place}/nodes_p4.pq")
stroke = gpd.read_parquet(f"../output/{place}/stroke_p3.pq")
blocks = gpd.read_parquet(f"../output/{place}/blocks_morphometric_p4.5.pq")

In [ ]:
stroke.plot(stroke.length,
                figsize=(15, 15),
                cmap="viridis_r",
                linewidth=.5,
                scheme="headtailbreaks"
               ).set_axis_off()

In [ ]:
stroke["stroke_length"] = stroke.length
stroke = stroke.drop(columns=["n_segments"])

In [ ]:
buildings_merged = buildings.merge(tessellation.drop(columns=['geometry']), on='uID', how='inner')
buildings_merged_centroids = buildings_merged
buildings_merged_centroids["geometry"] = buildings_merged_centroids['geometry'].centroid

In [ ]:
# Your latitude and longitude tuple (latitude, longitude)

# Transform the coordinates
centre_point_4326 = Point(lon, lat)
x_centre, y_centre = transform(4326, local_crs, lat, lon)
centre_point_local_crs = Point(x_centre, y_centre)

## Linear Regression

In [ ]:
raster_data = gpd.read_parquet(f"../output/{place}/raster_data.pq").to_crs(local_crs)

In [ ]:
raster_data

In [ ]:
# Create a 50 km buffer around the point
buffer_circle = gpd.GeoDataFrame({'geometry': [Point(lon, lat)]}, crs=4326).to_crs(local_crs).buffer(radius * 1000)

In [ ]:
# Get bounds
minx, miny, maxx, maxy = raster_data.total_bounds
min_dist = min([abs(minx-lonlat[1]), abs(miny-lonlat[0]), abs(maxx-lonlat[1]), abs(maxy-lonlat[0])])

# Create a GeoDataFrame with the specified point
gdf = gpd.GeoDataFrame(geometry=[Point(lonlat[0], lonlat[1])], crs=crs)

# The UTM zone number can be calculated from the longitude
utm_zone = int((lonlat[0] + 180) / 6) + 1
# Determine the hemisphere and assign the appropriate UTM CRS
if lonlat[1] >= 0:  # Northern Hemisphere
    local_utm_crs = f'EPSG:326{utm_zone:02d}'
else:  # Southern Hemisphere
    local_utm_crs = f'EPSG:327{utm_zone:02d}'

# Reproject to a local UTM CRS
gdf_utm = gdf.to_crs(local_utm_crs)

# Create a 50 km buffer around the point
buffer_circle = gdf_utm.buffer((radius) * 1000).to_crs(local_crs)  # Convert km to meters

In [ ]:
# # Create a plot
# fig, ax = plt.subplots(figsize=(10, 10))

# # Plot the first GeoDataFrame
# cells_in_circle.plot(ax=ax, color='blue', edgecolor='k', alpha=0.2)  # Adjust color and edgecolor as needed

# # Plot the second GeoDataFrame on the same axes
# buffer_circle.plot(ax=ax, color='red', alpha=0.2)  # Adjust color and alpha for transparency as needed

# tessellation.plot(ax=ax, color="green", alpha=0.4)

# # Optionally set axis limits or other plot properties
# # ax.set_xlim([xmin, xmax])
# # ax.set_ylim([ymin, ymax])

# ctx.add_basemap(ax, crs=local_crs,source=ctx.providers.CartoDB.Positron)

# plt.show()

In [ ]:
cells_in_circle = raster_data[raster_data["geometry"].within(buffer_circle.geometry[0])]


In [ ]:
cells_in_circle.plot()

In [ ]:
cells_in_circle

In [ ]:
# cells_in_circle.explore()

In [ ]:
# # Assuming cells_in_circle is a GeoDataFrame
# fig, ax = plt.subplots()
# cells_in_circle.plot(column='row', ax=ax, cmap='Blues')
# plt.show()

In [ ]:
buildings

In [ ]:
cells_in_circle.reset_index()[["geometry", "row", "col"]]

In [ ]:
buildings_joint = buildings.sjoin(cells_in_circle.reset_index()[["geometry", "row", "col"]], how='inner', op="intersects")

In [ ]:
blocks_joint = blocks.sjoin(cells_in_circle.reset_index()[["geometry", "row", "col"]], how='inner', op="intersects").drop(columns="index_right")

In [ ]:
blocks_joint

In [ ]:
# buildings_joint.explore()

In [ ]:
joint_gdf = tessellation.join(buildings_joint, on="uID", how="inner", lsuffix="_tess", rsuffix="_build").drop(["uID_tess"], axis=1)

In [ ]:
joint_gdf.set_geometry("geometry_build")

In [ ]:
tessellation_not_in_buildings = tessellation[~tessellation['uID'].isin(buildings_joint['uID'])]

In [ ]:
tessellation_not_in_buildings.plot()

In [ ]:
joint_gdf = joint_gdf.set_geometry("geometry_build")

In [ ]:
joint_gdf.to_parquet(f"../output/{place}/p5-buildings-output.pq")

In [ ]:
# # Create a copy of the joint_gdf to avoid modifying the original while iterating
# joint_gdf_copy = joint_gdf.copy()

# # List to hold the new rows
# new_rows = []

# for index, row in joint_gdf.iterrows():
#     for x in range(-1, 2):  # Adjusted range to include -1, 0, 1
#         for y in range(-1, 2):  # Adjusted range to include -1, 0, 1
#             if x == 0 and y == 0:  # Skip the current cell
#                 continue
#             else:
#                 # Copy the current row and modify 'row' and 'col'
#                 new_row = row.copy()
#                 new_row["row"] += x
#                 new_row["col"] += y
#                 new_rows.append(new_row)

# # Convert the list of new rows to a GeoDataFrame and concatenate it with the original
# new_rows_gdf = gpd.GeoDataFrame(new_rows)
# joint_gdf = pd.concat([joint_gdf_copy, new_rows_gdf], ignore_index=True)


In [ ]:
joint_gdf = joint_gdf.drop(columns=["geometry_tess", "geometry_build", "uID", "bID_build", "uID_build", "index_right"])

In [ ]:
joint_gdf.columns

In [ ]:
# Assuming 'gdf' is your GeoDataFrame
# Convert GeoDataFrame to Dask DataFrame
dask_df = dd.from_pandas(joint_gdf, npartitions=10)  # You can adjust the number of partitions

In [ ]:
# this is a function to compute a measure of diversity (Theil index)
def theil(y):
    y = np.array(y)
    n = len(y)
    plus = y + np.finfo('float').tiny * (y == 0)  # can't have 0 values
    yt = plus.sum(axis=0)
    s = plus / (yt * 1.0)
    lns = np.log(n * s)
    slns = s * lns
    t = sum(slns)
    return t

In [71]:
# Adjusted function to calculate metrics
def calculate_metrics(group):
    # Ensure group is treated as a DataFrame
    if isinstance(group, pd.Series):
        edges = group.to_frame().T
    
    results = {}
    for col in group.columns:
        # Exclude 'row' and 'col' from calculations
        if col not in ['row', 'col']:
            # Calculations as before
            q1, q3 = group[col].quantile([0.25, 0.75])
            iqr = q3 - q1
            within_iqr = group[col][(group[col] >= q1) & (group[col] <= q3)]
            mean_within_iqr = within_iqr.mean()

            p10, p90 = group[col].quantile([0.1, 0.9])
            within_percentiles = group[col][(group[col] >= p10) & (group[col] <= p90)]
            theil_index = theil(within_percentiles) if len(within_percentiles) > 0 else np.nan

            results[f'{col}_IQR'] = iqr
            results[f'{col}_MeanWithinIQR'] = mean_within_iqr
            results[f'{col}_Theil'] = theil_index

    return pd.Series(results)

# Apply the function to each group and convert to DataFrame
grouped =dask_df.groupby(["row", "col"])
metrics_df = grouped.apply(calculate_metrics).compute().reset_index()


In [72]:
blocks_joint.columns

Index(['bID', 'geometry', 'block_area', 'block_circularcompactness',
       'block_orientation', 'block_Neighbors', 'block_count', 'row', 'col'],
      dtype='object')

In [73]:
blocks_joint

,bID,geometry,block_area,block_circularcompactness,block_orientation,block_Neighbors,block_count,row,col
0,0,"POLYGON ((253481.044 668083.005, 253478.348 66...",343225.958675,0.560174,15.938277,0.003176,0.000472,163,170
0,0,"POLYGON ((253481.044 668083.005, 253478.348 66...",343225.958675,0.560174,15.938277,0.003176,0.000472,163,169
0,0,"POLYGON ((253481.044 668083.005, 253478.348 66...",343225.958675,0.560174,15.938277,0.003176,0.000472,163,168
0,0,"POLYGON ((253481.044 668083.005, 253478.348 66...",343225.958675,0.560174,15.938277,0.003176,0.000472,163,167
0,0,"POLYGON ((253481.044 668083.005, 253478.348 66...",343225.958675,0.560174,15.938277,0.003176,0.000472,163,166
...,...,...,...,...,...,...,...,...,...
8994,8994,"POLYGON ((268455.131 648423.317, 268452.339 64...",1168.516370,0.349636,5.841276,0.024733,0.000856,334,330
8995,8995,"POLYGON ((270836.599 653393.012, 270806.102 65...",705.960315,0.414315,42.861157,0.008436,0.001417,289,357
8995,8995,"POLYGON ((270836.599 653393.012, 270806.102 65...",705.960315,0.414315,42.861157,0.008436,0.001417,289,356
8995,8995,"POLYGON ((270836.599 653393.012, 270806.102 65...",705.960315,0.414315,42.861157,0.008436,0.001417,288,357


In [74]:
# Assuming 'gdf' is your GeoDataFrame
# Convert GeoDataFrame to Dask DataFrame
dask_blocks_df = dd.from_pandas(blocks_joint.drop(columns=["bID", "geometry"]), npartitions=10)  # You can adjust the number of partitions

grouped =dask_blocks_df.groupby(["row", "col"])
metrics_blocks_df = grouped.apply(calculate_metrics).compute().reset_index()

C:\Users\reube\AppData\Local\Temp\ipykernel_36264\3940449907.py:6: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  metrics_blocks_df = grouped.apply(calculate_metrics).compute().reset_index()


In [75]:
# Calculate the count of entries in each cell
cell_counts = dask_df.groupby(['row', 'col']).size().compute()
cell_counts_df = cell_counts.reset_index(name='building_count')

In [76]:
metrics_df = pd.merge(metrics_df, metrics_blocks_df, on=['row', 'col'], how='inner')

In [77]:
metrics_df = pd.merge(metrics_df, cell_counts_df, on=['row', 'col'], how='inner')

In [78]:
raster_data

,row,col,geometry,GHS_BUILT_H_100,GHS_BUILT_S_100,GHS_BUILT_V_100,GHS_LAND_100,GHS_POP_100
425,0,455,"POLYGON ((279328.852 685463.365, 279418.350 68...",0,21.0,94.0,8111,0.0
784,1,364,"POLYGON ((271185.384 685587.823, 271274.881 68...",0,145.0,363.0,10000,0.0
785,1,365,"POLYGON ((271274.881 685585.141, 271364.379 68...",0,1.0,3.0,10000,0.0
1295,3,281,"POLYGON ((263758.714 685592.876, 263848.211 68...",0,1109.0,2773.0,10000,3.0
1296,3,282,"POLYGON ((263848.211 685590.038, 263937.707 68...",0,890.0,2225.0,10000,2.0
...,...,...,...,...,...,...,...,...
67831,363,45,"POLYGON ((242845.021 646051.437, 242934.813 64...",0,1355.0,3388.0,9950,1.0
67832,363,46,"POLYGON ((242934.813 646048.197, 243024.606 64...",0,188.0,470.0,10000,0.0
67833,363,63,"POLYGON ((244461.290 645993.412, 244551.083 64...",0,2.0,5.0,10000,0.0
67834,363,64,"POLYGON ((244551.083 645990.206, 244640.876 64...",0,133.0,333.0,10000,0.0


In [79]:
raster_data = raster_data.set_index(["row", "col"])

In [80]:
metrics_df = metrics_df.dropna()

In [81]:
out_df = gpd.GeoDataFrame(pd.concat([metrics_df, raster_data], axis=1).reset_index(), geometry="geometry")
out_df = out_df.merge(cell_counts_df, on=['row', 'col'], how='left')

In [82]:
out_df = out_df.drop(columns=["geometry"], axis=1)
out_df = out_df.merge(cells_in_circle[["geometry", "row", "col"]], on=["row", "col"], how="inner")

In [83]:
out_df = gpd.GeoDataFrame(out_df, geometry='geometry')

In [84]:
# Function to flatten and clean column names (tuples or strings) from the GeoDataFrame
def clean_column_names(columns):
    clean_names = []
    for col in columns:
        if isinstance(col, tuple):
            clean_name = '_'.join(part.strip() for part in col)
        else:
            clean_name = col.strip()
        clean_names.append(clean_name)
    return clean_names

# Apply the cleaning function to the GeoDataFrame columns
out_df.columns = clean_column_names(out_df.columns)
out_df.columns.tolist()

['index',
 'row',
 'col',
 'tess_area_IQR',
 'tess_area_MeanWithinIQR',
 'tess_area_Theil',
 'tessellation_neighbours_IQR',
 'tessellation_neighbours_MeanWithinIQR',
 'tessellation_neighbours_Theil',
 'tess_convexity_IQR',
 'tess_convexity_MeanWithinIQR',
 'tess_convexity_Theil',
 'tess_covered_area_IQR',
 'tess_covered_area_MeanWithinIQR',
 'tess_covered_area_Theil',
 'tess_orientation_IQR',
 'tess_orientation_MeanWithinIQR',
 'tess_orientation_Theil',
 'tess_convexity_weight_IQR',
 'tess_convexity_weight_MeanWithinIQR',
 'tess_convexity_weight_Theil',
 'tess_covered_area_weight_IQR',
 'tess_covered_area_weight_MeanWithinIQR',
 'tess_covered_area_weight_Theil',
 'tess_orientation_weight_IQR',
 'tess_orientation_weight_MeanWithinIQR',
 'tess_orientation_weight_Theil',
 'tess_rea_theil_IQR',
 'tess_rea_theil_MeanWithinIQR',
 'tess_rea_theil_Theil',
 'tess_compactnessweightedaxis_IQR',
 'tess_compactnessweightedaxis_MeanWithinIQR',
 'tess_compactnessweightedaxis_Theil',
 'tess_compactnes

In [85]:
# out_df.explore()

In [86]:
out_df_expanded = out_df

In [87]:
for index, row in out_df.iterrows():
    geometry_list = [row.geometry]
    for x in range(-1, 2):  # Adjusted range to include -1, 0, 1
        for y in range(-1, 2):  # Adjusted range to include -1, 0, 1
            # Filter out_df for the specific row and column values
            filtered_df = out_df[(out_df["row"] == row.row + x) & (out_df["col"] == row.col + y)]
            # Check if the filtered_df is not empty
            if not filtered_df.empty:
                # Append the geometry of the first row from the filtered dataframe
                geometry_list.append(filtered_df.geometry.iloc[0])
    
    # Update the 'out_df_expanded' with the unified geometry
    out_df_expanded.loc[(out_df_expanded['row'] == row.row) & (out_df_expanded['col'] == row.col), 'geometry'] = unary_union(geometry_list)

In [88]:
def calculate_statistics(name, group, out_df):
    # Spatial join - associating each node with a polygon
    joined = gpd.sjoin(nodes, out_df, how='inner', op='intersects')

    # Creating a list to hold results
    results = []

    # Iterating through each unique polygon in the spatial join
    for polygon_id in joined['index_right'].unique():
        # Filter to include only nodes within the current polygon
        nodes_in_polygon = joined[joined['index_right'] == polygon_id]

        # Calculating statistics for each column
        stats = {'polygon_id': polygon_id}
        
        stats[f'nodes_count'] = nodes_in_polygon.iloc[0].count()
        
        for col in group.columns.drop(["nodeID", "geometry"]):
            if len(group[col]) == 1:
                break;
            # Exclude 'row' and 'col' from calculations
            if col not in ['row', 'col']:
                # Calculations as before
                q1, q3 = group[col].quantile([0.25, 0.75])
                iqr = q3 - q1
                within_iqr = group[col][(group[col] >= q1) & (group[col] <= q3)]
                mean_within_iqr = within_iqr.mean()

                p10, p90 = group[col].quantile([0.1, 0.9])
                within_percentiles = group[col][(group[col] >= p10) & (group[col] <= p90)]
                theil_index = theil(within_percentiles) if len(within_percentiles) > 0 else np.nan

                stats[f'{col}_IQR'] = iqr
                stats[f'{col}_MeanWithinIQR'] = mean_within_iqr
                stats[f'{col}_Theil'] = theil_index
                
        # Adding stats for this polygon to the results list
        results.append(stats)

    # Creating a DataFrame from the results
    stats_df = pd.DataFrame(results)

    # Ensuring that the merge key in out_df is a regular column (not an index)
    
    # out_df.explore()
    
    out_df['polygon_id'] = out_df.index

    # Merging the stats DataFrame with the out_df GeoDataFrame
    merged_df = out_df.merge(stats_df, on='polygon_id', how='left')
    
    merged_df = merged_df.drop(columns=["polygon_id"], axis = 1)

    return merged_df

In [89]:
out_df_expanded = calculate_statistics("nodes", nodes, out_df_expanded)

c:\Users\reube\anaconda3\envs\morphprocessor\Lib\site-packages\IPython\core\interactiveshell.py:3553: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
def calculate_line_statistics(name, group, out_df):
    # Spatial join - associating each line with polygons it intersects
    joined = gpd.sjoin(group, out_df, how='inner', op='intersects')

    # Creating a list to hold results
    results = []

    # Iterating through each unique polygon in the spatial join
    for polygon_id in joined['index_right'].unique():
        # Filter to include only lines intersecting the current polygon
        lines_in_polygon = joined[joined['index_right'] == polygon_id]
        
        # Calculating statistics for each column
        stats = {'polygon_id': polygon_id}
        
        stats[f'{name}_count'] = lines_in_polygon.iloc[0].count()
        
        # Columns to be dropped
        columns_to_drop = ['geometry', 'node_start', 'node_end', 'length']

        # Dropping columns that exist in the DataFrame from the list
        group = group.drop(columns=[col for col in columns_to_drop if col in group.columns])


        for col in group.columns:
            if len(group[col]) == 1:
                break;
            # Exclude 'row' and 'col' from calculations
            if col not in ['row', 'col']:
                # Calculations as before
                q1, q3 = group[col].quantile([0.25, 0.75])
                iqr = q3 - q1
                within_iqr = group[col][(group[col] >= q1) & (group[col] <= q3)]
                mean_within_iqr = within_iqr.mean()

                p10, p90 = group[col].quantile([0.1, 0.9])
                within_percentiles = group[col][(group[col] >= p10) & (group[col] <= p90)]
                theil_index = theil(within_percentiles) if len(within_percentiles) > 0 else np.nan

                stats[f'{col}_IQR'] = iqr
                stats[f'{col}_MeanWithinIQR'] = mean_within_iqr
                stats[f'{col}_Theil'] = theil_index
           
        # Adding stats for this polygon to the results list
        results.append(stats)

    # Creating a DataFrame from the results
    stats_df = pd.DataFrame(results)

    # Ensuring that the merge key in out_df is a regular column (not an index)
    out_df['polygon_id'] = out_df.index

    # Merging the stats DataFrame with the out_df GeoDataFrame
    merged_df = out_df.merge(stats_df, on='polygon_id', how="left")

    return merged_df

In [ ]:
edges = edges.drop(["mm_len", "node_start", "node_end"], axis=1)

In [ ]:
out_df_expanded = calculate_line_statistics("edges", edges, out_df_expanded)

In [ ]:
out_df_expanded = calculate_line_statistics("strokes", stroke, out_df_expanded)

In [ ]:
out_df_expanded = out_df_expanded.drop(columns=["geometry"], axis=1)
out_df = out_df_expanded.merge(cells_in_circle[["geometry", "row", "col"]], on=["row", "col"], how="inner")

In [ ]:
# Convert the merged DataFrame to a GeoDataFrame
geo_df = gpd.GeoDataFrame(out_df, geometry='geometry')

In [ ]:
geo_df = geo_df.drop(columns=["polygon_id"])

In [ ]:
# geo_df.explore()

In [ ]:
geo_df = geo_df.fillna(0)

In [ ]:
geo_df.to_parquet(f"../output/{place}/p5-grid-output.pq")

In [ ]:
# with rasterio.open(f'../output/{place}/p5.tif') as src:
#     # Get the transform object
#     transform = src.transform

#     # Extract west, north, and resolution (xres, yres)
#     west = transform.c
#     north = transform.f
#     xres = transform.a
#     yres = -transform.e  # yres is negative as raster origin is top-left


In [ ]:

# # Open the source raster to get the transform
# with rasterio.open(f'../output/{place}.tif') as src:
#     transform = src.transform

#     west = transform.c
#     north = transform.f
#     xres = transform.a
#     yres = -transform.e  # yres is negative as raster origin is top-left

# # Calculate raster dimensions
# n_rows = gdf['row'].max() - gdf['row'].min() + 1
# n_cols = gdf['col'].max() - gdf['col'].min() + 1

# # List of columns to be used as bands
# band_columns = [col for col in gdf.columns if col not in ['row', 'col', 'geometry']]

# # Initialize raster dataset
# with rasterio.open(f"../output/{place}/grid_with_summary_stats.tif", 'w', 
#     driver='GTiff', 
#     height=n_rows, 
#     width=n_cols, 
#     count=len(band_columns), 
#     dtype=str(gdf[band_columns[0]].dtype), 
#     crs=gdf.crs, 
#     transform=transform
# ) as dst:
#     for i, column in enumerate(band_columns, start=1):
#         # Create an empty array filled with NaNs (or use a different no-data value)
#         if column == 'building_height':
#             # For 'building_height', initialize with zeros
#             band_data = np.zeros((n_rows, n_cols))
#         else:
#             # For other columns, initialize with NaNs
#             band_data = np.full((n_rows, n_cols), np.nan)

#         # Update the array with data from the GeoDataFrame
#         for _, row in gdf.iterrows():
#             row_idx, col_idx = row['row'] - gdf['row'].min(), row['col'] - gdf['col'].min()
#             band_data[row_idx, col_idx] = row[column]

#         dst.write(band_data, i)

In [ ]:
# gdf['distance_to_centre'] = gdf['geometry'].distance(centre_point_local_crs)

In [ ]:
# new_gdf = gdf.drop(columns=["geometry", "row", "col"]).dropna()

In [ ]:
# # Response variable
# y = new_gdf['distance_to_centre']

# # Feature variables (excluding the response variable)
# X = new_gdf.drop(columns=['distance_to_centre'])

# # Standardize the features
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# # Initialize and fit the Linear Regression model
# model = LinearRegression()
# model.fit(X_scaled, y)

# # Coefficients and Intercept
# coefficients = model.coef_
# intercept = model.intercept_

# # Predictions
# predictions = model.predict(X_scaled)

# # Number of observations and number of features
# n = len(y)
# p = X.shape[1]

# # Residual Sum of Squares and Total Sum of Squares
# rss = np.sum((y - predictions) ** 2)
# tss = np.sum((y - np.mean(y)) ** 2)

# # R-squared and Adjusted R-squared
# r_squared = 1 - (rss / tss)
# adjusted_r_squared = 1 - ((1 - r_squared) * (n - 1) / (n - p - 1))

# # Standard Error of the coefficients
# mse = rss / (n - p - 1)
# se = np.sqrt(np.diagonal(mse * np.linalg.inv(np.dot(X_scaled.T, X_scaled))))

# # t-statistics and p-values for coefficients
# t_stats = coefficients / se
# p_values = [2 * (1 - stats.t.cdf(np.abs(t), n - p - 1)) for t in t_stats]

# # F-statistic
# f_statistic = (r_squared / p) / ((1 - r_squared) / (n - p - 1))

# def get_significance_asterisks(p):
#     if p < 0.001:
#         return '***'
#     elif p < 0.01:
#         return '**'
#     elif p < 0.05:
#         return '*'
#     else:
#         return ''

# # Create a summary DataFrame
# summary_df = pd.DataFrame({
#     'Coefficient': coefficients,
#     'Standard Error': se,
#     't value': t_stats,
#     'p value': p_values,
#     'Significance': [get_significance_asterisks(p) for p in p_values]
# }, index=X.columns)
# summary_df.loc['Intercept'] = [intercept, np.nan, np.nan, np.nan, '']

# # Print summary statistics
# print(f"Number of Observations: {n}")
# print(f"R-squared: {r_squared}")
# print(f"Adjusted R-squared: {adjusted_r_squared}")
# print(f"F-statistic: {f_statistic}\n")

In [ ]:
# summary_df.to_string()

In [ ]:
# # Assuming 'summary_df' is your DataFrame
# # Set the display options
# pd.set_option('display.max_rows', None)  # Replace None with a specific number if the DataFrame is very large
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)  # Adjust the width for better readability if necessary
# pd.set_option('display.max_colwidth', None)  # Adjust column width to show full content of each cell

In [ ]:
# summary_df

In [ ]:
# summary_df.to_parquet(f"../output/{place}/summary_df.pq")

In [ ]:
local_crs = 27700
place = "Glasgow"
lat = 55.86421405612109
lng = -4.251846930489373
country = "UK"
crs=4326
radius=100
